<a href="https://colab.research.google.com/github/tanaymukherjee/Deep-Learning-with-PyTorch/blob/master/10_Generative_Adversarial_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Networks

## Import packages and libraries

In [46]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

## Device configuration

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper-parameters

In [48]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

## Create a directory if not exists

In [49]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

## Generative Adversarial Networks

### Image processing

In [50]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

### MNIST dataset

In [51]:
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

### Data Loader

In [52]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

### Discriminator

In [53]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

### Generator

In [54]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

### Device setting

In [55]:
D = D.to(device)
G = G.to(device)

## Binary cross entropy loss and optimizer

In [56]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

## Start training

In [57]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0309, g_loss: 4.6184, D(x): 0.99, D(G(z)): 0.02
Epoch [0/200], Step [400/600], d_loss: 0.0899, g_loss: 5.1055, D(x): 0.97, D(G(z)): 0.05
Epoch [0/200], Step [600/600], d_loss: 0.0362, g_loss: 5.6525, D(x): 0.98, D(G(z)): 0.02
Epoch [1/200], Step [200/600], d_loss: 0.0662, g_loss: 4.7554, D(x): 1.00, D(G(z)): 0.06
Epoch [1/200], Step [400/600], d_loss: 0.0914, g_loss: 4.3014, D(x): 0.97, D(G(z)): 0.05
Epoch [1/200], Step [600/600], d_loss: 0.1168, g_loss: 4.5805, D(x): 0.95, D(G(z)): 0.04
Epoch [2/200], Step [200/600], d_loss: 0.0801, g_loss: 6.1143, D(x): 0.96, D(G(z)): 0.01
Epoch [2/200], Step [400/600], d_loss: 0.1498, g_loss: 4.4607, D(x): 0.92, D(G(z)): 0.04
Epoch [2/200], Step [600/600], d_loss: 0.2612, g_loss: 3.7164, D(x): 0.91, D(G(z)): 0.10
Epoch [3/200], Step [200/600], d_loss: 0.2154, g_loss: 4.9457, D(x): 0.91, D(G(z)): 0.07
Epoch [3/200], Step [400/600], d_loss: 0.2857, g_loss: 3.1585, D(x): 0.95, D(G(z)): 0.13
Epoch [3/200], Step [

## Save the model checkpoint

In [58]:
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')